<a href="https://colab.research.google.com/github/Mohammad-Faiz-2005/Apnacollege-demo/blob/main/cat_dog_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
import os
from google.colab import files

# Create a .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Upload the kaggle.json file
uploaded = files.upload()

# Get the filename from the uploaded dictionary
if uploaded:
    uploaded_filename = list(uploaded.keys())[0]
    # Move the uploaded kaggle.json to the .kaggle directory
    !mv "{uploaded_filename}" ~/.kaggle/kaggle.json

    # Set permissions (important for security)
    !chmod 600 ~/.kaggle/kaggle.json
    print(f"Successfully uploaded and configured {uploaded_filename} to ~/.kaggle/")
else:
    print("No file was uploaded. Please make sure to upload your kaggle.json file.")

Saving kaggle (3).json to kaggle (3).json
Successfully uploaded and configured kaggle (3).json to ~/.kaggle/


In [ ]:
!kaggle datasets download -d arpitjain007/dog-vs-cat-fastai
!unzip dog-vs-cat-fastai.zip

Streaming output truncated to the last 5000 lines.
  inflating: dogscats/train/dogs/dog.7066.jpg  
  inflating: dogscats/train/dogs/dog.7067.jpg  
  inflating: dogscats/train/dogs/dog.7068.jpg  
  inflating: dogscats/train/dogs/dog.7069.jpg  
  inflating: dogscats/train/dogs/dog.707.jpg  
  inflating: dogscats/train/dogs/dog.7070.jpg  
  inflating: dogscats/train/dogs/dog.7071.jpg  
  inflating: dogscats/train/dogs/dog.7072.jpg  
  inflating: dogscats/train/dogs/dog.7073.jpg  
  inflating: dogscats/train/dogs/dog.7074.jpg  
  inflating: dogscats/train/dogs/dog.7075.jpg  
  inflating: dogscats/train/dogs/dog.7076.jpg  
  inflating: dogscats/train/dogs/dog.7077.jpg  
  inflating: dogscats/train/dogs/dog.7078.jpg  
  inflating: dogscats/train/dogs/dog.7079.jpg  
  inflating: dogscats/train/dogs/dog.708.jpg  
  inflating: dogscats/train/dogs/dog.7080.jpg  
  inflating: dogscats/train/dogs/dog.7081.jpg  
  inflating: dogscats/train/dogs/dog.7082.jpg  
  inflating: dogscats/train/dogs/dog.70

In [ ]:
pip install tensorflow numpy


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "/content/dogscats/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_data = val_gen.flow_from_directory(
    "/content/dogscats/valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

print(train_data.class_indices)
# {'cats': 0, 'dogs': 1}


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
{'cats': 0, 'dogs': 1}


In [ ]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")
])


In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
class_weight = {
    0: 1.0,   # Cat
    1: 1.5    # Dog (boost)
}


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weight
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 343s 449ms/step - accuracy: 0.9485 - loss: 0.1522 - val_accuracy: 0.9780 - val_loss: 0.0579
Epoch 2/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 293s 407ms/step - accuracy: 0.9705 - loss: 0.0819 - val_accuracy: 0.9865 - val_loss: 0.0356
Epoch 3/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 290s 403ms/step - accuracy: 0.9732 - loss: 0.0783 - val_accuracy: 0.9825 - val_loss: 0.0443
Epoch 4/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 282s 392ms/step - accuracy: 0.9786 - loss: 0.0681 - val_accuracy: 0.9710 - val_loss: 0.0895
Epoch 5/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 298s 414ms/step - accuracy: 0.9783 - loss: 0.0642 - val_accuracy: 0.9805 - val_loss: 0.0500
Epoch 6/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 291s 405ms/step - accuracy: 0.9790 - loss: 0.0645 - val_accuracy: 0.9840 - val_loss: 0.0393
Epoch 7/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 291s 405ms/step - accuracy: 0.9780 - loss: 0.0624 - val_accuracy: 0.9830 - val_loss: 0.0457
Epoch 8/10
719/719 ━━━━━━━━━━━━━━━━━━━━ 293s 407ms/step - accuracy: 0.9785 -

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    class_weight=class_weight
)


Epoch 1/5
719/719 ━━━━━━━━━━━━━━━━━━━━ 330s 434ms/step - accuracy: 0.9189 - loss: 0.2379 - val_accuracy: 0.9735 - val_loss: 0.0820
Epoch 2/5
719/719 ━━━━━━━━━━━━━━━━━━━━ 292s 406ms/step - accuracy: 0.9678 - loss: 0.0949 - val_accuracy: 0.9745 - val_loss: 0.0728
Epoch 3/5
719/719 ━━━━━━━━━━━━━━━━━━━━ 296s 412ms/step - accuracy: 0.9753 - loss: 0.0784 - val_accuracy: 0.9775 - val_loss: 0.0619
Epoch 4/5
719/719 ━━━━━━━━━━━━━━━━━━━━ 292s 407ms/step - accuracy: 0.9765 - loss: 0.0724 - val_accuracy: 0.9800 - val_loss: 0.0463
Epoch 5/5
719/719 ━━━━━━━━━━━━━━━━━━━━ 294s 408ms/step - accuracy: 0.9825 - loss: 0.0584 - val_accuracy: 0.9780 - val_loss: 0.0543


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.makedirs("/content/drive/MyDrive/cat_dog_project", exist_ok=True)

model.save("/content/drive/MyDrive/cat_dog_project/cat_dog_model.keras")


In [ ]:
model.save("cat_dog_model.h5")

In [ ]:
import numpy as np
from tensorflow import keras

img = keras.utils.load_img("/content/dogscats/valid/cats/cat.10071.jpg", target_size=(IMG_SIZE,IMG_SIZE))
img = keras.utils.img_to_array(img) / 255.0
img = np.expand_dims(img, axis=0)

pred = model.predict(img)

print("Dog" if pred[0][0] >= 0.5 else "Cat")
print(pred[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Cat
6.071244e-06
